In [546]:
import numpy as np
import matplotlib.pyplot as plt
import random
import enum

# Задание 1

Решить линейную регрессию — найти прямую, которая описывает зависимость

N - количество осей (размерность) <br>
M - количество точек <br>
minValue - минимальное значение для иксов <br>
maxValue - максимальное значение для иксов

In [547]:
N = 1
M = 30
minValue = 0
maxValue = 5
EPS = 1e-5
K = 1e-3

class TypeOfOptimisation(enum.Enum):
    momentum = 0
    nesterov = 1
    RMSProp = 2
    AdaGrad = 3
    none = 4


In [548]:
def yRegression(B, X):
    return np.sum(X * B)

B_reg = []
for i in range(N + 1):
    bi = random.randint(-maxValue, maxValue)
    B_reg.append(bi)
B_reg = np.array(B_reg, dtype='float64')
print(B_reg)
XM = []
YM = []
YR = []
for i in range(M):
    X = [1]
    for j in range(N):
        X.append(i)
    X = np.array(X)
    rand_delta = random.randint(-maxValue, maxValue)
    Y_act = yRegression(B_reg, X) + 0.5 * rand_delta
    YM.append(Y_act)
    XM.append(np.array(X))
XM = np.array(XM, dtype='float64')
YM = np.array(YM)

def runSGD(typeOfOptimisation, withNormalization):
    B_start = np.array([0] * (N + 1), dtype='float64')
    YR = np.array([0] * M, dtype='float64')
    if withNormalization:
        X_norm = ZNormalization(XM)
        Y_norm = (YM - np.mean(YM)) / np.std(YM)
        BA_start = (B_reg - np.mean(B_reg)) / np.std(B_reg)
        BF_norm, \
        errors_norm, \
        YF_norm, \
        iterations = gradientDescent(1,
                                     X_norm,
                                     Y_norm,
                                     YR,
                                     B_start,
                                     TypeOfOptimisation.none,
                                     withNormalization=False)

        print(errors_norm[0], errors_norm[len(errors_norm) - 1])
        print("actual coefficients:     ", BA_start)
        print("calculated coefficients: ", BF_norm)
        print("number of iterations:    ", iterations)
        drawGraph(X_norm, Y_norm, YF_norm)
    else:
        B_final, \
        errors, \
        YF, \
        iterations = gradientDescent(1,
                                     XM,
                                     YM,
                                     YR,
                                     B_start,
                                     typeOfOptimisation,
                                     withNormalization=True)

        print(errors[0], errors[len(errors) - 1])
        print("actual coefficients:     ", B_reg)
        print("calculated coefficients: ", B_final)
        print("number of iterations:    ", iterations)
        drawGraph(XM, YM, YF)

[5. 5.]


In [549]:
def squaredError(Y_act, Y_reg):
    return np.sum((Y_act - Y_reg)**2) / M

def stoppingCriteria(previous, current, withNormalization):
    if withNormalization:
        eps = EPS * 1e-3
    else:
        eps = EPS
    return abs(previous - current) < eps

def gradient(nums, XM, YR, YM):
    grad = []
    for i in range(N + 1):
        xi = 0
        for j in range(len(nums)):
            xi += XM[nums[j]][i] * (YR[nums[j]] - YM[nums[j]])
        xi *= 2
        grad.append(xi)
    return np.array(grad)

def gradientDescent(batch, XM, YM, YR, B_reg, typeOfOptimisation, withNormalization):
    previousError = 0
    previousGradient = np.array([0] * (N + 1))
    errors = []
    iterations = 0
    while True:
        nums = set()
        while len(nums) != batch:
            x = random.randint(0, M - 1)
            if x not in nums:
                nums.add(x)

        nums = np.array(list(nums))
        grad = gradient(nums, XM, YR, YM)

        if typeOfOptimisation == TypeOfOptimisation.momentum:
            grad += gamma * previousGradient

        if withNormalization:
            k = K * 2
        else:
            k = K
        B_reg -= k * grad

        for i in range(M):
            YR[i] = yRegression(B_reg, XM[i])
        if stoppingCriteria(previousError, squaredError(YM, YR), withNormalization):
            break
        previousError = squaredError(YM, YR)
        previousGradient = grad
        errors.append(previousError)
        iterations += 1
    return B_reg, errors, YR, iterations

def drawGraph(XM, YM, YF):
    fig = plt.figure()
    fig.set_size_inches(10, 10)
    X = XM.T[1]
    plt.scatter(X, YM)
    plt.plot(X, YF, color='red')
    plt.show()

runSGD(TypeOfOptimisation.none, withNormalization=False)

KeyboardInterrupt: 

# Задание 2

In [ ]:
def ZNormalization(XM):
    X = XM.T
    for i in range(1, N + 1):
        X[i] = (X[i] - np.mean(X[i])) / np.std(X[i])
    return X.T

# print(XM)
runSGD(TypeOfOptimisation.none, withNormalization=True)

# Задание 3

### Метод моментов

In [ ]:
gamma = 0.9

runSGD(TypeOfOptimisation.momentum, withNormalization=False)
runSGD(TypeOfOptimisation.momentum, withNormalization=True)
